In [33]:
import twitter
import os
import yaml
import re
import time
import tweepy
import pandas as pd
from textblob import TextBlob
from collections import Counter
import pickle

In [2]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/api_credentials.yml')))

# Try Tweepy

In [36]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
	"""Download the last 3240 tweets from a user. Do text processign to remove URLs and the retweets from a user.
	Adapted from https://gist.github.com/yanofsky/5436496"""
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(credentials['twitter']['consumer_key'], credentials['twitter']['consumer_secret'],)
	auth.set_access_token(credentials['twitter']['token'], credentials['twitter']['token_secret'])
	api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:

		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
	print(f'Finished getting tweets for {screen_name}')
	cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in alltweets] # remove urls
	cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions 
	cleaned_text = [re.sub(r'RT.*','', i, flags=re.MULTILINE) for i in cleaned_text] # delete the retweets
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, cleaned_text[idx]] for idx,tweet in enumerate(alltweets)]
	
# 	#write the csv	
# 	with open(f'../data/raw/svb_founders/{company}:{screen_name}_tweets.csv', 'w') as f:
# 		writer = csv.writer(f)
# 		writer.writerow(["id","created_at","text"])
# 		writer.writerows(outtweets)
	
	return pd.DataFrame(outtweets,columns=["id","created_at","text"])


In [8]:
tweets_df = get_all_tweets('ericries')

getting tweets before 864843472724934659
...400 tweets downloaded so far
getting tweets before 855108316174852096
...600 tweets downloaded so far
getting tweets before 848723992840097791
...800 tweets downloaded so far
getting tweets before 842780620669435904
...1000 tweets downloaded so far
getting tweets before 838066835924766719
...1200 tweets downloaded so far
getting tweets before 833477351585390591
...1400 tweets downloaded so far
getting tweets before 826235426260201471
...1600 tweets downloaded so far
getting tweets before 795656380245909503
...1798 tweets downloaded so far
getting tweets before 791448562747965439
...1998 tweets downloaded so far
getting tweets before 784882493237243903
...2198 tweets downloaded so far
getting tweets before 776075851750342655
...2398 tweets downloaded so far
getting tweets before 763146623585165311
...2598 tweets downloaded so far
getting tweets before 749008182748459007
...2798 tweets downloaded so far
getting tweets before 742383680212086784


In [15]:
def scrub_text(tweets_df, return_bag_of_words = False, num_chars = 10):
    """Takes in a tweets DF and returns a list of sentences .
    Also creates the bag of words from the text to be used at a later time.
    num_chars specifies the minimum number of characters per sentence"""
    bag_of_words = None
    all_sentences = []
    for row in tweets_df.iterrows():
        blob = TextBlob(row[1]['text'])
        blob = blob.lower()
        blobl = blob.strip()# remove whitespace
        for sent in blob.sentences: ## append each sentence
            if len(sent) < num_chars: # sentences need to have at least ten characters
                pass
            else:
                all_sentences.append(str(sent)+" ")
        tokens = blob.tokenize()
        if bag_of_words == None:
            bag_of_words =  Counter(tokens)
        else:
            words = Counter(tokens)
            for k,v in words.items():
                if k in bag_of_words:
                    bag_of_words[k]+=v
                else:
                    bag_of_words[k]=v
    if return_bag_of_words == True:
        return all_sentences, bag_of_words
    else:
        return all_sentences
                
    

In [27]:
vc1_pitchbook_df = pd.read_csv( "../data/processed/PitchBook_CA_VCInvest=1.csv")

In [46]:
vc1_pitchbook_dfbook_dfbook_dfbook_df.columns

Index(['Unnamed: 0', 'Company ID', 'Description', 'Company Name',
       'HQ Post Code', 'Primary Industry Code', 'Primary Contact',
       'Year Founded', 'Active Investors', 'Growth Rate', 'Size Multiple',
       'Last Financing Date', 'Last Financing Size',
       'Last Financing Deal Type 2 ', 'Company Name.1',
       'Majestic Referring Domains', 'Facebook Likes', 'Twitter Followers',
       'Employees', 'Total Raised', 'VC_invested', 'Twitter_Username'],
      dtype='object')

In [28]:
vc1_pitchbook_df

,Unnamed: 0,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,Growth Rate,...,Last Financing Size,Last Financing Deal Type 2,Company Name.1,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested,Twitter_Username
0,0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,"Sinclair Broadcast Group, Sinclair Digital Ven...",0.000000,...,4.42,Series B,ZypMedia,115.5,1135.0,51.0,12.0,6.820,1,mgoldman
1,2,60184-09,Developer of non-invasive sensors for laborato...,Zyomed,91011,Diagnostic Equipment,Sandeep Gulati,2007.0,"Khosla Ventures, Samsung Electronics",0.000000,...,10.61,Series B2,Zyomed,2.0,1135.0,1098.0,27.0,23.110,1,Sandeepgulati16
2,3,97130-35,Developer of procedures to optimize microbes f...,Zymergen,94608,Other Business Products and Services,Zach Serber,2013.0,"5 Prime Ventures, AME Cloud Ventures, Andy War...",0.632376,...,130.00,Series B,Zymergen,288.0,1135.0,911.0,200.0,176.140,1,ZachSerber
3,4,52852-15,Provider of a channel data management cloud pl...,Zyme,94065,Media and Information Services (B2B),Chandran Sankaran,2004.0,"Artiman Ventures, Asiabio Zyme Solutions, Susq...",4.139861,...,100.00,NaN,Zyme,330.0,197.0,1300.0,501.0,13.100,1,chandransank
4,6,62229-34,"Developer of animated, interactive learning ma...",Zyante,95031,Publishing,Smita Bakshi,2012.0,"Bialla Venture Partners, National Science Foun...",0.078953,...,4.00,Series A,Zyante,115.5,612.0,267.0,32.0,4.000,1,sbaksh
5,7,52214-95,Developer of music creation software designed ...,Zya,91302,Multimedia and Design Software,Alberto Lomeli,2008.0,"Individual Investor, Intel Capital, Liberty Gl...",0.192032,...,4.26,NaN,Zya,123.0,2269.0,1098.0,51.0,44.860,1,alomeli23
6,8,117491-32,Provider of interactive fitness entertainment ...,Zwift,90802,Entertainment Software,Eric Min,2014.0,"COLOPL, Max Levchin, Novator Partners, Samchul...",0.143940,...,27.00,Series A,Zwift,471.0,158746.0,24569.0,70.0,45.000,1,ShinhwaCompany
7,10,103593-34,Developer of a mobile-based browsing applicati...,Zurf,94107,Social/Platform Software,Shahin Shadfar,2013.0,"Bobby Yazdani, Geekdom SF, Signatures Capital",-0.088942,...,6.00,NaN,Zurf,25.0,166.0,127.0,6.0,11.185,1,sshadfar
8,11,51251-95,Provider of an on-demand subscription billing ...,Zuora,94404,Other Software,Tyler Sloat,2007.0,"Benchmark Capital, BlackRock Private Equity Pa...",0.179027,...,6.00,NaN,Zuora,2031.0,3827.0,17692.0,631.0,250.000,1,tylersloat
9,12,55336-69,Provider of a mobile and Web-based listing ser...,Zumper,94108,Social/Platform Software,Anthemos Georgiades,2012.0,"Andreessen Horowitz, Breyer Capital, CrunchFun...",0.314371,...,17.65,Series B,Zumper,3167.0,57829.0,8081.0,45.0,39.250,1,anthemos


In [45]:
username_errors = []
for idx,row in enumerate(vc1_pitchbook_df.iterrows()):
    total_rows = len(vc1_pitchbook_df)
    
    founder = row[1]['Primary Contact']
    company = row[1]['Company Name']
    twitter_username = row[1]['Twitter_Username']
    try:
        tweets = get_all_tweets(twitter_username )
        scrubbed_tweets = scrub_text(tweets)
        with open(f"../data/raw/founders_tweets/vc_invest=1/{company}-{founder}-{twitter_username}", "wb") as output_file:
            pickle.dump(scrubbed_tweets, output_file)
    except: # not authorized to see this user's timeline
        username_errors.append(founder) ## eventually drop these usernames

    print(f"{idx/total_rows:%} percent finished")
    
 

Finished getting tweets for mgoldman
0.000000% percent finished
Finished getting tweets for Sandeepgulati16
0.155280% percent finished
Finished getting tweets for ZachSerber
0.310559% percent finished
Finished getting tweets for chandransank
0.465839% percent finished
Finished getting tweets for sbaksh
0.621118% percent finished
Finished getting tweets for alomeli23
0.776398% percent finished
Finished getting tweets for ShinhwaCompany
0.931677% percent finished
Finished getting tweets for sshadfar
1.086957% percent finished
Finished getting tweets for tylersloat
1.242236% percent finished
Finished getting tweets for anthemos
1.397516% percent finished
Finished getting tweets for cross2roads
1.552795% percent finished
Finished getting tweets for JeopardyJulia
1.708075% percent finished
Finished getting tweets for macdguitar
1.863354% percent finished
Finished getting tweets for davecarter30
2.018634% percent finished
Finished getting tweets for pvpbrussels
2.173913% percent finished
Fin

Finished getting tweets for EAsteveWaterloo
19.720497% percent finished
Finished getting tweets for wardnyt
19.875776% percent finished
Finished getting tweets for SaMeHSaRHaN2
20.031056% percent finished
Finished getting tweets for GinaAnnGarcia
20.186335% percent finished
Finished getting tweets for Amirdanesh
20.341615% percent finished
Finished getting tweets for yanivm
20.496894% percent finished
Finished getting tweets for awh
20.652174% percent finished
20.807453% percent finished
Finished getting tweets for ParinDalal
20.962733% percent finished
Finished getting tweets for stevendegennaro
21.118012% percent finished
21.273292% percent finished
Finished getting tweets for davyjacobsfans
21.428571% percent finished
Finished getting tweets for ColeyBarko
21.583851% percent finished
Finished getting tweets for sashapoljak
21.739130% percent finished
Finished getting tweets for sdubois
21.894410% percent finished
Finished getting tweets for workman
22.049689% percent finished
Finish

Finished getting tweets for dsmitheconomics
40.527950% percent finished
Finished getting tweets for MFDell7
40.683230% percent finished
Finished getting tweets for vitalusche
40.838509% percent finished
Finished getting tweets for sundarkrishhh
40.993789% percent finished
41.149068% percent finished
Finished getting tweets for itsBhumiBhutani
41.304348% percent finished
Finished getting tweets for eMilwaukee
41.459627% percent finished
41.614907% percent finished
Finished getting tweets for CFeistel
41.770186% percent finished
Finished getting tweets for thefuntastic
41.925466% percent finished
Finished getting tweets for repdavidscott
42.080745% percent finished
Finished getting tweets for durrenshen
42.236025% percent finished
Finished getting tweets for ThreeGuys1Book
42.391304% percent finished
Finished getting tweets for DerekDeMeyer
42.546584% percent finished
Finished getting tweets for allenlau
42.701863% percent finished
Finished getting tweets for martinezshopper
42.857143% p

Finished getting tweets for quinnjacobson
60.714286% percent finished
Finished getting tweets for plaikind
60.869565% percent finished
61.024845% percent finished
Finished getting tweets for hidhussain
61.180124% percent finished
Finished getting tweets for joewaltman
61.335404% percent finished
Finished getting tweets for JuanJoseEstruch
61.490683% percent finished
Finished getting tweets for dj_fitzdoodcast
61.645963% percent finished
Finished getting tweets for beccacolbert
61.801242% percent finished
Finished getting tweets for borjarossell
61.956522% percent finished
Finished getting tweets for Anki4uever
62.111801% percent finished
62.267081% percent finished
Finished getting tweets for byuric
62.422360% percent finished
Finished getting tweets for Prakash10529714
62.577640% percent finished
Finished getting tweets for iam_charles_
62.732919% percent finished
Finished getting tweets for en0ch85
62.888199% percent finished
Finished getting tweets for BrightenGodfrey
63.043478% per

81.366460% percent finished
Finished getting tweets for mmcnallycycling
81.521739% percent finished
Finished getting tweets for wesley_kuo
81.677019% percent finished
Finished getting tweets for coachchadparks
81.832298% percent finished
Finished getting tweets for MilesandBarr
81.987578% percent finished
Finished getting tweets for jessicarichman
82.142857% percent finished
Finished getting tweets for johnfaieta
82.298137% percent finished
82.453416% percent finished
Finished getting tweets for patrickbunk
82.608696% percent finished
Finished getting tweets for tangowhisky26
82.763975% percent finished
Finished getting tweets for mobisimo
82.919255% percent finished
Finished getting tweets for samjadallah
83.074534% percent finished
Finished getting tweets for torbjorn_l
83.229814% percent finished
Finished getting tweets for chrisdralla
83.385093% percent finished
Finished getting tweets for kvedati
83.540373% percent finished
Finished getting tweets for apre2016
83.695652% percent f

In [39]:
scrubbed_tweets 

['#flyboy #flygirl victory party. ',
 'a great day at #im703wisconsin with the most amazing training… ',
 ' beyond amazing shot! ',
 ' ready for the mud fest!!! ',
 ' thank you for your leadership and counsel in helping  become the great president he is. ',
 'our team members continue to do great things in our communities! ',
 'senior developer/workflow automation - plymouth, mn - tcf bank ',
 'great for mlb! ',
 '#temporarycubsfan ',
 'mission accomplished. ',
 '#avoidthe8run8th #gocubsgo ',
 '#avoidthe8run8th #gocubsgo ',
 'unbelievable recovery run from the  marathon... the fall colors were amazing and the… ',
 'the stage is set for the 2016 twin cities marathon! ',
 'one more sleep! ',
 'this will be my 4th marathon… ',
 '\n\n#presidentialdebate #debate ',
 ' first look at our race day forecast is out! ',
 "if this delivers as promised,  we're in for a great day! ",
 'tcf introduces chip debit card with contactless payment capability ',
 'still the one: peterson keeps a diversified

# Python -Twitter

In [124]:
text = api.GetUserTimeline(screen_name='ashady',count =300, include_rts=False)

In [125]:
cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls

In [127]:
cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions

In [128]:
cleaned_text

[" Nice work Eric. Love the analysis. We're building the same using AI. Starting making picks this month  demo day",
 'Flying  and really impressed by the crew. Great service and super friendly!',
 'I just published “Which 500 Startups Demo Day companies does our AI algorithm like?” ',
 'When will AI replace venture capitalists? ',
 'Venture Capital: Are any VC firms using machine learning or AI to help in deal sourcing? ',
 'NYU Stern School of Business  featuring  Stern pride goes both ways :) \n',
 'What are the common traits of successful entrepreneurs? ',
 '6 Traits of Successful Founders ',
 '    thanks Chris!! Means a lot that you like it!',
 'I just published “The Personalities of Successful Founders” ',
 ' ',
 'I just donated to  to help children in need. You can help too!: ',
 ' which one?',
 'Attending a cool session on NLP and machine learning at  ',
 'Complex #DataViz by  on  untangles   business ties ',
 ' yes photos pls!',
 'Just donated ',
 'What it means to stand again

In [129]:
api.GetFollowers(screen_name='ashady')

[User(ID=59840147, ScreenName=kvasnic),
 User(ID=868973790386151424, ScreenName=EsraaEl51255104),
 User(ID=288497184, ScreenName=JTattersfield),
 User(ID=870175789546958848, ScreenName=ADo7a93),
 User(ID=24961055, ScreenName=mattbiehl),
 User(ID=744150240136044544, ScreenName=djkingassassin),
 User(ID=711273343396278272, ScreenName=JaydaBF),
 User(ID=351100725, ScreenName=Dylan_Bostic),
 User(ID=864015366946775040, ScreenName=RehamMohamedAh7),
 User(ID=296320946, ScreenName=SAPTradeGeek),
 User(ID=866383033292771328, ScreenName=TurkiBinA2),
 User(ID=865124920598179840, ScreenName=PeaceHackMENA),
 User(ID=799012517129269248, ScreenName=MuhammedEzzat10),
 User(ID=1879978364, ScreenName=ryanhintze),
 User(ID=805559119566565376, ScreenName=ask1angel),
 User(ID=3784808057, ScreenName=bouigoudal),
 User(ID=25217355, ScreenName=_moustafa),
 User(ID=266420236, ScreenName=treyptrsn),
 User(ID=2707473229, ScreenName=DreamsdotBuild),
 User(ID=556259902, ScreenName=OmarAbdElRhmaan),
 User(ID=86024

In [108]:
def twitter_text(username):
    """Return the last 200 tweets' text from the given username"""
    text = api.GetUserTimeline(screen_name=username,count =300, include_rts=False)
    cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls
    cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions
    return cleaned_text

In [130]:
twitter_text('olivercameron')

['It took ~4 years but they did it! ',
 'Surely radar and cameras is preferable to LIDAR? Primarily for range &amp; cost: ',
 '   Definitely!',
 '  did all the good work there 👌',
 'We recently rebuilt our entire  controls stack for silky smooth rides. Lots of related good stuff in here! ',
 ' Great decision!  and the team are building a world-changing product.',
 'Awesome opportunity. ',
 '😱 ',
 '   You can in the MKZ, not in the Fusion.',
 'A template for building a great company, courtesy of . ',
 ' Pumped to see what you build next! Just my personal opinion, but I would try to cure cancer with dinosaurs.',
 '😶  ',
 '    ',
 '   is the exact opposite, prefers Antarctic winds in the car.',
 'To be clear: this is a bug in the  app, not the actual car temperature ☀️ ',
 '🤔 ',
 'Insane growth! ',
 'Fun visit to  earlier. Mapping (and all that comes with it) is a huge pain-point in autonomous vehicles… ',
 'Awesome interview. ',
 '      Can you imagine?',
 'I should mention, we plan to o

In [ ]:
def twitter_followers(username):
    """REturn the number of followers for the given user. Need to sleep to not exceed the rate limit."""
    for num in range(1_000_000):
        if num %500 ==0:
            time.sleep(5)
            